## Extract phenotype data for all protein coding genes from Wormbase

In [3]:
import pandas as pd
import time

wormbase_version="WS293"

gene_ids_df = pd.read_csv(f"./wormbase_data/c_elegans.PRJNA13758.{wormbase_version}.geneIDs.csv") 

# Extract only the protein coding genes
protein_coding_genes_df = gene_ids_df[gene_ids_df["Gene_Type"].isin(["protein_coding_gene", "gene"])]

# Expect we have 21,506 genes to process
assert len(protein_coding_genes_df) == 21_506

# UTILITY FUNCTIONS
# Track the time to make a function call
def formatted_elapsed_time(start,end=None):
    minute=60
    hour  =60 * minute

    if end == None:
        end = time.time()
    total_seconds = end - start
    hours = total_seconds // hour
    minutes = (total_seconds % hour) // minute
    seconds = (total_seconds % hour) % minute
    return f'Time: {hours=} {minutes=} {seconds=:.2f}'


In [4]:
import csv

# Given a phenotype_dict and a phenotype_list add only new (unique) phenotypes from the list to the dictionary
def unique_phenotypes(phenotype_dict, phenotype_list):
    for item in phenotype_list:
        wbpt_id = item['wbpt_id']
        wbpt_name = item['wbpt_name']
        
        evidence_allele = item.get('evidence_allele', None)
        evidence_rnai = item.get('evidence_rnai', None)
        evidence = ""
        if evidence_allele:
            evidence = 'allele'
        elif evidence_rnai:
            evidence = evidence_rnai
            
        if wbpt_id not in phenotype_dict:
            phenotype_dict[wbpt_id] = {'name':wbpt_name, 'evidence':evidence}
    return phenotype_dict

# Given a phenotype_list return a dictionary with unique phenotypes and a count of 
# the occurrences of that phenotype in the list
def count_phenotypes(phenotype_list):
    phenotype_counts = {}
    for phenotype in phenotype_list:
        wbpt_id = phenotype['wbpt_id']
        if wbpt_id in phenotype_counts:
            phenotype_counts[wbpt_id] += 1
        else:
            phenotype_counts[wbpt_id] = 1
            
    return phenotype_counts

# Write the list of uniques phenotypes to a file
def phenotypes_to_csv(phenotype_dict, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["ID", "Name","Evidence"])  # header
        for wbpt_id, wbpt_data in sorted(phenotype_dict.items()):
            name = wbpt_data.get('name','')
            evidence = wbpt_data.get('evidence','')
            writer.writerow([wbpt_id, name, evidence])

# Write the phenotypes for each gene to a file
def gene_phenotypes_to_csv(gene_phenotypes, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["wormbase_id", "phenotype", "count"])  # header
        for gene, phenotypes in sorted(gene_phenotypes.items()):
            for phenotype, count in phenotypes.items():
                writer.writerow([gene, phenotype, count])



## Wormbase API Calls

* Get the phenotype data for all protein coding genes
* Get all unique phenotype descriptions

<span style="color:red">Note: Executing the below cell will take approximately 6 Minutes to run.</span>

In [4]:
from pub_worm.wormbase.wormbase_api import WormbaseAPI

start_time = time.time()

# Set the API Class to get phenotype data from Wormbase
wormbase_api = WormbaseAPI("field", "gene", "phenotype")

genes_to_process = protein_coding_genes_df['Wormbase_Id'].tolist()

# This is a Multi-process call using 10 CPUs
wormbase_data_results = wormbase_api.get_wormbase_data_cpu(genes_to_process, 10)
print(formatted_elapsed_time(start_time))



Check if you have a connection!! | Retry- 1 | Response msg- <urlopen error [Errno 60] Operation timed out>
Check if you have a connection!! | Retry- 1 | Response msg- <urlopen error [Errno 60] Operation timed out>
Time: hours=0.0 minutes=5.0 seconds=23.53


In [ ]:
gene_phenotypes = {}
phenotype_dict = {}
count=0
for wormbase_data_result in wormbase_data_results:
    wormbase_id, phenotype_list_dict = next(iter(wormbase_data_result.items()))    
    #If we has a phenotype_list we process it if we have an empty dict we skip
    if 'phenotype_list' in phenotype_list_dict:
        # If phenotype_list_dict['phenotype_list'] is a dict it means we only have 1 result
        # but we will wrap it in a list to make processing easier
        if isinstance(phenotype_list_dict['phenotype_list'], dict):
            phenotype_list = [phenotype_list_dict['phenotype_list']]
        else:
            phenotype_list = phenotype_list_dict['phenotype_list']
            
        phenotype_dict = unique_phenotypes(phenotype_dict, phenotype_list)
        phenotype_counts_dict = count_phenotypes(phenotype_list)
        gene_phenotypes[wormbase_id]= phenotype_counts_dict

    
phenotypes_to_csv(phenotype_dict, f"wormbase_data/phenotypes_{wormbase_version}.csv")
gene_phenotypes_to_csv(gene_phenotypes, f"wormbase_data/gene_phenotypes_{wormbase_version}.csv")


# Appendix

In [ ]:
!pip install --upgrade  pub_worm